In [3]:
from geopy.geocoders import Bing
f = open('api_key_bing.txt', 'r')
api = f.read()
f.close()
geolocator = Bing(api)
location = geolocator.geocode("Санкт-Петербург, Обводном Днепропетровской")
print((location.latitude, location.longitude))
coord = location.latitude, location.longitude

(59.93318176269531, 30.306039810180664)


In [1]:
from geopy.geocoders import Yandex
f = open('api_key.txt', 'r')
api = f.read()
f.close()
geolocator = Yandex(api)
location = geolocator.geocode("Санкт-Петербург, Типанова Космонавтов")
print((location.latitude, location.longitude))
coord = location.latitude, location.longitude

(59.852872, 30.352223)


In [10]:
import pandas as pd
data = pd.read_csv('data/new_with_street.csv')
data = data.dropna()
data = data.reset_index()
data.drop(['index'], axis=1, inplace=True)
start_date = '2019-11-01'
end_date = '2019-12-01'
mask = (data['date'] >= start_date) & (data['date'] <= end_date)
df = data.loc[mask]

In [12]:
df.shape

(845, 4)

In [18]:
df = df[:1]

In [19]:
from folium.plugins import MarkerCluster
import folium
from geopy.geocoders import Yandex
f = open('api_key.txt', 'r')
api = f.read()
f.close()
geolocator = Yandex(api)
ad = []
spb = [59.939095, 30.315868]
map = folium.Map(location=spb, zoom_start = 10)
marker_cluster = MarkerCluster().add_to(map)
for i in list(df.index.values):
    st = df['pavlov'][i]
    if st != '':
        id = df['id'][i]
        addr = f'Санкт-Петербург, {st}'
        location = geolocator.geocode(addr)
        if location is None:
            continue
        if round(location.latitude, 6) == spb[0] and round(location.longitude, 6) == spb[1]:
            continue
        folium.Marker(location=[location.latitude, location.longitude], popup = folium.Popup(f'<a href="https://vk.com/spb_today?w=wall-68471405_{id}"target="_blank">link to news in VK</a>'), icon=folium.Icon(color = 'red', icon='info-sign')).add_to(marker_cluster)
map.save("map1.html")